In [1]:
import pandas as pd
import numpy as np
import bqplot
import ipywidgets
import matplotlib.pyplot as plt
from bqplot import market_map
import traitlets
from IPython.display import clear_output, display
from knapsack_problem_solver import *

In [2]:
trucks = [transit, e350, box_truck, platform]
branches = ['ZAGA']

In [3]:
branch_options = ipywidgets.Dropdown(
    options = branches,
    description = 'Branch: '
)
truck_options = ipywidgets.Dropdown(
    options = trucks,
    description = 'Truck: '
)
style = {'description_width': 'initial'}
empty_len = ipywidgets.FloatSlider(
    value = 0.5,
    min = 0,
    max = 1,
    step = 0.05,
    description = 'Empty Length: ',
    style = style
)
empty_wid = ipywidgets.FloatSlider(
    value = 0.5,
    min = 0,
    max = 1,
    step = 0.05,
    description = 'Empty Width: ',
    style = style
)
empty_hei = ipywidgets.FloatSlider(
    value = 0.5,
    min = 0,
    max = 1,
    step = 0.05,
    description = 'Empty Height: ',
    style = style
)
title = ipywidgets.HTML(' \
                       <!DOCTYPE html> \
                        <html> \
                        <head> \
                        <style> \
                        .center { \
                            margin: 0 auto; \
                            max-width: 50%; \
                            padding: 1.3rem; \
                        } \
 \
                        .title-bar { \
                            background-color: #8497b0; \
                            color: #ffffff; \
                        } \
 \
                        </style> \
                        </head> \
                        <body> \
                        <div class="title-bar"> \
                            <div class="center"> \
                              <h3>Truck Distribution Recommendation System</h3> \
                            </div> \
                        </div> \
                        </body> \
                        </html> \
                     ')

In [4]:
theme_blue = '#8497b0'

layout_center_list = ipywidgets.Layout()
layout_center_list.width = '50%'
layout_center_list.margin= '0.1% auto'

whole_layout_center = ipywidgets.Layout()
whole_layout_center.width = '100%'
whole_layout_center.margin = '0 auto'

layout_button = ipywidgets.Layout()
layout_button.width = '50%'
layout_button.margin = '1% auto'

two_opposite_side = ipywidgets.Layout()
two_opposite_side.display = 'flex'
two_opposite_side.justify_content = 'space-between'

add_top_margin = ipywidgets.Layout()
add_top_margin.margin = '5% 0% 0.1%'

In [5]:
button = ipywidgets.Button(description='Calculate', button_style='success')
output = ipywidgets.Output()
sol_out = ipywidgets.Output()
viz = ipywidgets.Output()
sol_df = None
pie_chart_colors = [theme_blue, '#85c5a6', '#cbebb5']
                    
def on_dataView_clicked(b):
    with viz:
        clear_output(True)
        labels = list(set(sol_df['Material Class']))
        sizes = []
        for label in labels:
            tmp_df = sol_df[sol_df['Material Class'] == label].reset_index()
            profit = 0
            for i in range(len(tmp_df)):
                profit += tmp_df.iloc[i]['Quantity'] * tmp_df.iloc[i]['Unit Profit']
            sizes.append(profit)
        fig1, ax1 = plt.subplots()
        ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
                shadow=True, startangle=0, colors=pie_chart_colors)
        ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
        ax1.set_title('Profit Contribution by Material Class')
        plt.show()

def on_button_clicked(b):
    with output:
        clear_output(True)
        global viz
        viz = ipywidgets.Output()
        viz.layout = add_top_margin
        display(ipywidgets.HTML(
            value = '<h4>Calculating Recommendation...</h4>'
        ))
        solver = KnapsackSolver(KnapsackSolverParser(), truck_options.value, [empty_len.value, empty_wid.value, empty_hei.value], True)
        best_value, best_sol, transformer = solver.solve()
        global sol_df
        sol_df = transformer.transform_solution(best_sol)
        
        clear_output(True)
        
        value_display = ipywidgets.HTML(
            value = '<div align="center" style="border:1px solid {1}"> \
                <h4 style="color: black">Profit when 100% sold: <b>${0}</b></h4> \
            </div>'.format(best_value, theme_blue)
        )
        
        intro_sol_out = ipywidgets.HTML(
            value = '<h4>Recommendation of products : </h4>'
        )
        intro_sol_out.layout = add_top_margin
        
        show_viz_button = ipywidgets.Button(description='Data Analytics View', button_style='success')
        show_viz_button.on_click(on_dataView_clicked)
        show_viz_button.style.button_color = theme_blue
        show_viz_button.layout = add_top_margin
        
        intro_line = ipywidgets.HBox([intro_sol_out, show_viz_button])
        intro_line.layout = two_opposite_side
        
        with sol_out:
            clear_output(True)
            display(sol_df)
        display(ipywidgets.VBox([value_display, intro_line, sol_out, viz]))

button.on_click(on_button_clicked)

spaces_box = ipywidgets.VBox([empty_len, empty_wid, empty_hei])
user_operation_box = ipywidgets.VBox([branch_options, truck_options, spaces_box, button])

branch_options.layout = layout_center_list
truck_options.layout = layout_center_list
empty_hei.layout = layout_center_list
empty_len.layout = layout_center_list
empty_wid.layout = layout_center_list

button.style.button_color  = theme_blue
button.layout = layout_button

output.layout = layout_center_list

display(ipywidgets.VBox([title, user_operation_box, output]))

In [6]:
sol_df

In [7]:
for i in range(len(sol_df)):
    print(sol_df.iloc[i]['GMR'])

TypeError: object of type 'NoneType' has no len()